In [1]:
!pip install -q transformers
!pip install -q seqeval

     |████████████████████████████████| 1.9MB 18.3MB/s 
     |████████████████████████████████| 890kB 54.0MB/s 
     |████████████████████████████████| 3.2MB 53.9MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 


In [2]:
!git clone https://github.com/Ryzhtus/master-thesis

Cloning into 'master-thesis'...
remote: Enumerating objects: 2192, done.
remote: Counting objects: 100% (2192/2192), done.
remote: Compressing objects: 100% (2126/2126), done.
remote: Total 2192 (delta 110), reused 2145 (delta 63), pack-reused 0
Receiving objects: 100% (2192/2192), 2.62 MiB | 18.47 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [3]:
cd master-thesis

/content/master-thesis


In [4]:
from transformers import BertTokenizer
import torch

import warnings
warnings.filterwarnings('ignore')

In [5]:
from named_entity_recognition.dataset import DatasetNER, create_dataset_and_dataloader
from named_entity_recognition.model import BertNER
from named_entity_recognition.train import train_model, test_model

In [6]:
TOKENIZER = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
DEVICE = 'cuda' if torch.cuda.is_available else 'cpu'
EPOCHS = 4
BATCH_SIZE = 16

In [7]:
train_dataset, train_dataloader = create_dataset_and_dataloader('conll', "data/conll2003/train.txt", BATCH_SIZE, TOKENIZER)
eval_dataset, eval_dataloader = create_dataset_and_dataloader('conll', "data/conll2003/valid.txt", BATCH_SIZE, TOKENIZER)
test_dataset, test_dataloader = create_dataset_and_dataloader('conll', "data/conll2003/test.txt", BATCH_SIZE, TOKENIZER)

In [8]:
eval_dataset.idx2tag = train_dataset.idx2tag
eval_dataset.tag2idx = train_dataset.tag2idx
test_dataset.idx2tag = train_dataset.idx2tag
test_dataset.tag2idx = train_dataset.tag2idx

In [9]:
import torch.optim as optim
import torch
import torch.nn as nn

classes = len(train_dataset.ner_tags)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = BertNER(classes).to(device) 
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)

EPOCHS = 4

train_model(model, criterion, optimizer, train_dataloader, eval_dataloader, train_dataset.tag2idx, train_dataset.idx2tag, device, None, EPOCHS)
test_model(model, criterion, test_dataloader, train_dataset.tag2idx, train_dataset.idx2tag, device)

[1 / 4] Train: Loss = 0.20401, F1-score = 86.00%, Repeated Entities Accuracy = 76.32%: 100%|██████████| 878/878 [02:49<00:00,  5.17it/s]
[1 / 4] Eval : Loss = 0.10649, F1-score = 93.41%, Repeated Entities Accuracy = 91.15%: 100%|██████████| 204/204 [00:10<00:00, 19.40it/s]
[2 / 4] Train: Loss = 0.08431, F1-score = 94.67%, Repeated Entities Accuracy = 92.80%: 100%|██████████| 878/878 [02:48<00:00,  5.20it/s]
[2 / 4] Eval : Loss = 0.08161, F1-score = 94.98%, Repeated Entities Accuracy = 93.28%: 100%|██████████| 204/204 [00:10<00:00, 19.57it/s]
[3 / 4] Train: Loss = 0.05522, F1-score = 96.55%, Repeated Entities Accuracy = 96.05%: 100%|██████████| 878/878 [02:48<00:00,  5.22it/s]
[3 / 4] Eval : Loss = 0.09063, F1-score = 94.76%, Repeated Entities Accuracy = 92.59%: 100%|██████████| 204/204 [00:10<00:00, 19.51it/s]
[4 / 4] Train: Loss = 0.04396, F1-score = 97.23%, Repeated Entities Accuracy = 97.52%: 100%|██████████| 878/878 [02:48<00:00,  5.22it/s]
[4 / 4] Eval : Loss = 0.08714, F1-score =